In [10]:
# Surgiu problemas com os dados que estavam fora da verificação e estavam sendo transformados em json antes disso.
# R: Corrigi criando a estrutura SE. 

# Ocorreu dos movimentos dos pokémons terem mais de 4 linhas, fazendo a tabela dar erro por não ter espaço. 
# R: Resolvido após colocar para apenas utilizar os primeiros 4 ataques encontrados usando o for.

# Enquanto estava mexendo no algoritmo, ele estava comparando 1 pokémon com o outro ao invés de comparar todos. 
# R: Resolvido colocando pra IA verificar todos os pokémons e analisar quem tem os 4 status mais altos.

# Surgiu problemas quanto a quantidade de dados que o algoritmo puxava da API. 
# R: Corrigido após limitar a quantidade de dados puxados. 

In [11]:
#Projeto enviado para o GitHub

#https://github.com/Fellipe0408/Projeto-PokeAPI

In [12]:
import requests as r
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [13]:
# Variável para limitar a busca de Pokémon
pokemon_limit = 10

In [14]:
# Função para buscar todos os Pokémon e seus status
def fetch_all_pokemon(limit):
    url_list = f'https://pokeapi.co/api/v2/pokemon?offset=0&limit={limit}'
    resp_list = r.get(url_list)
    data_list = resp_list.json()
    
    all_pokemon_stats = {}
    
    for pokemon in data_list['results']:
        name = pokemon['name']
        url = pokemon['url']
        resp = r.get(url)
        
        if resp.status_code == 200:
            data = resp.json()
            values = [v['base_stat'] for v in data['stats']]
            all_pokemon_stats[name] = values
            
    return all_pokemon_stats

# Busca todos os Pokémon
all_pokemon_stats = fetch_all_pokemon(pokemon_limit)

# Saída para exibir os resultados do Pokémon selecionado
output_selected = widgets.Output()


In [15]:
# Função para simular o uso de um LLM para sugestões de treinamento
def suggest_training_with_llm(status, values):
    suggestions = []
    
    for stat, value in zip(status, values):
        if stat == 'hp' and value < 50:
            suggestions.append("Treinamento de Resiliência: Enfrente Pokémon resistentes para aumentar seus HP.")
        elif stat == 'attack' and value < 50:
            suggestions.append("Treinamento de Força: Treine contra Pokémon com baixa defesa.")
        elif stat == 'defense' and value < 50:
            suggestions.append("Treinamento de Defesa: Melhore sua defesa enfrentando Pokémon com ataque alto.")
        elif stat == 'special-attack' and value < 50:
            suggestions.append("Treinamento de Ataque Especial: Treine contra Pokémon vulneráveis a ataques especiais.")
        elif stat == 'special-defense' and value < 50:
            suggestions.append("Treinamento de Defesa Especial: Enfrente Pokémon que usam ataques especiais.")
        elif stat == 'speed' and value < 50:
            suggestions.append("Treinamento de Agilidade: Melhore sua velocidade enfrentando Pokémon mais lentos.")
    
    return suggestions

In [16]:
dropdown = widgets.Dropdown(
        options=list(all_pokemon_stats.keys()),  # Opções baseadas nos Pokémon disponíveis
        description='Escolha o Pokémon:',
)

In [17]:
# Função para buscar informações de um Pokémon e exibir em um DataFrame
def fetch_pokemon_info(selected_pokemon):
    url = f'https://pokeapi.co/api/v2/pokemon/{selected_pokemon}'
    resp = r.get(url)
    
    with output_selected:
        clear_output()  
        if resp.status_code == 200:  # Verificar se está funcionando
            data = resp.json()
            
            # Coleta habilidades, tipos, status, valores e movimentos
            abilities = [i['ability']['name'] for i in data['abilities']]
            types = [t['type']['name'] for t in data['types']]
            status = [s['stat']['name'] for s in data['stats']]
            values = [v['base_stat'] for v in data['stats']]
            movimentos = [m['move']['name'] for i, m in enumerate(data['moves']) if i < 4]

            # Garante que as listas tenham o mesmo tamanho
            max_len = max(len(abilities), len(types), len(status), len(values), len(movimentos))
            abilities += [''] * (max_len - len(abilities))
            types += [''] * (max_len - len(types))
            status += [''] * (max_len - len(status))
            values += [''] * (max_len - len(values))
            movimentos += [''] * (max_len - len(movimentos))

            # Cria o DataFrame com habilidades, tipos, status, valores e movimentos
            df_data = pd.DataFrame({
                'Tipo': types,    
                'Habilidades': abilities,
                'Status': status,
                'Valor': values,
                'Movimentos': movimentos
            })

            # Cria um gráfico de pizza para os valores dos status
            plt.figure(figsize=(8, 6))  # Ajusta o tamanho da figura

            # Adiciona um subplot para o gráfico
            plt.subplot(121)
            plt.pie(values, labels=status, startangle=140)
            plt.title(f'Status de {selected_pokemon}')
            plt.axis('equal')

            # Adiciona o DataFrame ao segundo subplot
            plt.subplot(122) 
            plt.axis('off') 
            table = plt.table(cellText=df_data.values, colLabels=df_data.columns, cellLoc='center', loc='right')
            table.auto_set_font_size(False)
            table.set_fontsize(10)  # Ajusta o tamanho da fonte
            table.scale(2.0, 2.0)  # Ajusta o tamanho da tabela

            plt.show()  # Mostra a figura

            # Sugere treinamento com base nos status usando o "LLM" (IA)
            training_suggestions = suggest_training_with_llm(status, values)
            if training_suggestions:
                print("Sugestões de treinamento:")
                for suggestion in training_suggestions:
                    print(f"- {suggestion}")
            else:
                print("Todos os status estão em níveis adequados para batalhar.")
        else:
            print('Não existe este Pokémon')

In [18]:
# Função para comparar e exibir os 5 Pokémon mais fortes
def display_top_pokemon():
    with output_top_five:
        clear_output()
        if len(all_pokemon_stats) >= 2:  # Deve haver pelo menos 2 Pokémon para comparação
            strong_pokemon_list = []
            for pokemon1, stats1 in all_pokemon_stats.items():
                count_stronger = 0
                for pokemon2, stats2 in all_pokemon_stats.items():
                    if pokemon1 != pokemon2:
                        # Compara os stats
                        greater_count = sum(1 for stat1, stat2 in zip(stats1, stats2) if stat1 > stat2)
                        if greater_count >= 4: 
                            count_stronger += 1
                strong_pokemon_list.append((pokemon1, count_stronger))
            
            # Ordena os Pokémon
            strong_pokemon_list.sort(key=compare_strength, reverse=True)

            print("Os 5 Pokémon mais fortes são:")
            for pokemon, count in strong_pokemon_list[:5]:
                print(f"{pokemon.capitalize()}")

        else:
            print("Selecione mais Pokémon para comparação.")

# Exibi os 5 Pokémon mais fortes
output_top_five = widgets.Output()

# Função de comparação para ordenar Pokémon
def compare_strength(item):
    return item[1]

# Função chamada quando o dropdown muda
def on_dropdown_change(change):
    selected_value = change['new']
    fetch_pokemon_info(selected_value)  # Chama a função para exibir informações do Pokémon selecionado
    display_top_pokemon()  # Atualiza a lista dos 5 Pokémon mais fortes


# Adiciona o observador ao dropdown
dropdown.observe(on_dropdown_change, names='value')

# Exibe os widgets
display(dropdown, output_selected, output_top_five)

Dropdown(description='Escolha o Pokémon:', options=('bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmel…

Output()

Output()